In [49]:
import pandas as pd
import unicodedata
import re
from thefuzz import process
from typing import List

In [50]:
pd.options.display.max_columns = 55
pd.options.display.max_rows = 20

In [51]:
def to_string(x):
    return str(x)

In [52]:
from deep_translator import GoogleTranslator

def remove_accents(input_str):
    nfkd_form = unicodedata.normalize('NFKD', input_str)
    only_ascii = nfkd_form.encode('ASCII', 'ignore')
    if only_ascii.decode('utf-8') == ' ':
        print("Translating: " + input_str)
        nfkd_form = unicodedata.normalize('NFKD', GoogleTranslator(source='auto', target='en').translate(input_str))
        only_ascii = nfkd_form.encode('ASCII', 'ignore')
        return only_ascii.decode('utf-8')
    else:
        return only_ascii.decode('utf-8')

In [53]:
def remove_alphanumeric(input_str):
    return re.sub(r'\W+', '', input_str).lower()

In [54]:
def best_matching_name(name, name_list):
    result = process.extractOne(name, name_list, score_cutoff=80)
    return result[0] if result else None

In [55]:
player_stats = pd.read_csv("C:\\Users\\soere\\Bachelorarbeit\\datasets\\raw_data\\player_stats.csv")

C:\Users\soere\AppData\Local\Temp\ipykernel_19644\2869168751.py:1: DtypeWarning: Columns (7) have mixed types. Specify dtype option on import or set low_memory=False.
  player_stats = pd.read_csv("C:\\Users\\soere\\Bachelorarbeit\\datasets\\raw_data\\player_stats.csv")


In [56]:
transfers = pd.read_csv("C:\\Users\\soere\\Bachelorarbeit\\datasets\\transfers.csv")
transfers = transfers[transfers["Aufnehmende Liga"].isin(["Premier League","Bundesliga","LaLiga","Ligue 1","Serie A"])]
transfers = transfers[transfers["Saison"] <= 23]

In [57]:
fifa = pd.read_csv("C:\\Users\\soere\\Bachelorarbeit\\datasets\\raw_data\\fifa_stats.csv")

C:\Users\soere\AppData\Local\Temp\ipykernel_19644\2874764412.py:1: DtypeWarning: Columns (108) have mixed types. Specify dtype option on import or set low_memory=False.
  fifa = pd.read_csv("C:\\Users\\soere\\Bachelorarbeit\\datasets\\raw_data\\fifa_stats.csv")


In [58]:
fifa.drop(columns=['player_url', 'fifa_update', 'update_as_of',
 'player_positions', 'value_eur', 'wage_eur', 'age', 'dob', 'height_cm',
 'weight_kg', 'club_team_id', 'league_id','league_name', 'league_level',
 'club_position', 'club_jersey_number', 'club_loaned_from',
 'club_joined_date', 'club_contract_valid_until_year', 'nationality_id',
 'nationality_name', 'nation_team_id', 'nation_position',
 'nation_jersey_number', 'real_face', 'release_clause_eur', 'player_tags',
 'player_traits', 'ls', 'st','rs', 'lw', 'lf', 'cf', 'rf', 'rw', 'lam', 'cam', 'ram',
 'lm', 'lcm', 'cm', 'rcm', 'rm', 'lwb', 'ldm', 'cdm', 'rdm', 'rwb', 'lb', 'lcb', 'cb',
 'rcb', 'rb', 'gk'],
  axis=1, inplace=True)

In [59]:
fifa["fifa_version"] = fifa["fifa_version"] - 1

In [60]:
fifa = fifa[fifa["fifa_version"] >= 18]

In [61]:
fifa["fifa_version"] = fifa["fifa_version"].apply(lambda x: int(x))

In [62]:
fifa["club_name"] = fifa["club_name"].apply(to_string)

Correcting values

In [63]:
fifa.rename(columns={"fifa_version":"Saison"},inplace=True)
fifa["fifa_name"] = fifa["long_name"].apply(remove_accents)
fifa["fifa_name"] = fifa["fifa_name"].apply(remove_alphanumeric)
fifa["fifa_club"] = fifa["club_name"].apply(remove_accents)
fifa["fifa_club"] = fifa["fifa_club"].apply(remove_alphanumeric)

Translating: 손흥민 孙兴慜
Translating: 김민재 金敏在
Translating: 鎌田 大地


KeyboardInterrupt: 

Lade schon übersetze fifa datei ein

In [ ]:
#fifa.to_csv("C:\\Users\\soere\\Bachelorarbeit\\datasets\\raw_data\\fifa_translated.csv",index=False)

In [64]:
fifa = pd.read_csv("C:\\Users\\soere\\Bachelorarbeit\\datasets\\raw_data\\fifa_translated.csv")

In [65]:
fifa

,Saison,long_name,overall,potential,club_name,preferred_foot,weak_foot,skill_moves,international_reputation,work_rate,body_type,pace,shooting,passing,dribbling,defending,physic,attacking_crossing,attacking_finishing,attacking_heading_accuracy,attacking_short_passing,attacking_volleys,skill_dribbling,skill_curve,skill_fk_accuracy,skill_long_passing,skill_ball_control,movement_acceleration,movement_sprint_speed,movement_agility,movement_reactions,movement_balance,power_shot_power,power_jumping,power_stamina,power_strength,power_long_shots,mentality_aggression,mentality_interceptions,mentality_positioning,mentality_vision,mentality_penalties,mentality_composure,defending_marking_awareness,defending_standing_tackle,defending_sliding_tackle,goalkeeping_diving,goalkeeping_handling,goalkeeping_kicking,goalkeeping_positioning,goalkeeping_reflexes,goalkeeping_speed,fifa_name,fifa_club
0,23,Kylian Mbappé Lottin,91,94,Paris Saint Germain,Right,4,5,5,High/Low,Unique,97.0,90.0,80.0,92.0,36.0,78.0,78,94,73,86,84,93,80,69,71,92,97,97,93,93,82,90,88,88,77,83,64,38,93,83,84,88.0,26,34,32,13,5,7,11,6,NaN,kylianmbappelottin,parissaintgermain
1,23,Erling Braut Haaland,91,94,Manchester City,Left,3,3,5,High/Medium,Unique,89.0,93.0,66.0,80.0,45.0,88.0,47,96,83,77,90,79,77,62,53,82,82,94,76,94,72,94,93,76,93,86,87,43,96,74,84,87.0,38,47,29,7,14,13,11,7,NaN,erlingbrauthaaland,manchestercity
2,23,Kevin De Bruyne,91,91,Manchester City,Right,5,4,5,High/Medium,Unique,72.0,88.0,94.0,87.0,65.0,78.0,95,85,55,94,83,86,92,83,94,92,72,72,74,92,78,92,72,88,74,92,75,66,88,95,83,88.0,66,70,53,15,13,5,10,13,NaN,kevindebruyne,manchestercity
3,23,Lionel Andrés Messi Cuccittini,90,90,Inter Miami,Left,4,4,5,Low/Low,Unique,80.0,87.0,90.0,94.0,33.0,64.0,83,89,60,91,86,96,93,93,90,93,87,74,91,88,95,83,71,70,68,90,44,40,91,92,75,96.0,20,35,24,6,11,15,14,8,NaN,lionelandresmessicuccittini,intermiami
4,23,Karim Benzema,90,90,Al Ittihad,Right,4,4,5,Medium/Medium,Normal (170-185),79.0,88.0,83.0,87.0,39.0,78.0,75,91,90,89,88,87,82,73,76,91,78,79,77,92,72,87,85,82,82,81,63,39,92,90,85,90.0,43,24,18,13,11,5,5,7,NaN,karimbenzema,alittihad
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
111578,18,David Norman Jr.,47,61,Vancouver Whitecaps,Right,3,2,1,Medium/Medium,Lean (185+),62.0,35.0,47.0,47.0,41.0,61.0,38,25,40,55,31,49,34,36,51,41,57,66,56,47,52,54,63,55,67,34,51,36,49,45,43,40.0,47,35,48,8,6,12,6,9,NaN,davidnormanjr,vancouverwhitecaps
111579,18,Christoph Ehlich,47,59,Unterhaching,Right,3,2,1,High/Medium,Lean (170-185),63.0,42.0,38.0,50.0,41.0,56.0,39,40,45,46,42,46,35,29,22,49,65,62,68,44,70,44,69,61,59,41,38,36,44,38,47,47.0,40,42,42,13,12,11,15,12,NaN,christophehlich,unterhaching
111580,18,Shandon Baptiste,47,59,Oxford United,Right,3,2,1,Medium/Medium,Normal (170-185),59.0,32.0,46.0,48.0,45.0,47.0,36,27,47,50,32,47,35,38,44,44,56,61,55,54,56,48,61,43,45,25,54,49,29,54,37,37.0,42,44,46,5,13,10,15,14,NaN,shandonbaptiste,oxfordunited
111581,18,Evan Tweed,47,58,Derry City,Right,3,2,1,Medium/Medium,Normal (170-185),56.0,36.0,46.0,44.0,42.0,50.0,37,34,49,55,27,40,31,30,46,43,59,54,54,53,66,47,64,44,50,30,52,45,35,49,40,43.0,39,39,48,6,11,9,5,8,NaN,evantweed,derrycity


In [66]:
fifa["Saison"] = fifa["Saison"].apply(lambda x: str(x))

In [67]:
player_stats.rename(columns={"Verein":"Abgebender Verein"},inplace=True)
player_stats["Saison"] = player_stats["Saison"].apply(lambda x: str(x+1))
player_stats["stats_name"] = player_stats["Spieler"].apply(remove_accents)
player_stats["stats_name"] = player_stats["stats_name"].apply(remove_alphanumeric)
player_stats["stats_club"] = player_stats["Abgebender Verein"].apply(remove_accents)
player_stats["stats_club"] = player_stats["stats_club"].apply(remove_alphanumeric)

In [68]:
transfers["Saison"] = transfers["Saison"].apply(to_string)
transfers["ID"] = transfers["Spieler"].apply(remove_accents)
transfers["ID"] = transfers["ID"].apply(remove_alphanumeric)
transfers["clubID"] = transfers["Abgebender Verein"].apply(remove_accents)
transfers["clubID"] = transfers["clubID"].apply(remove_alphanumeric)

Constructing a matching names dataset (only needed once, then stored as csv)

In [69]:
#stats_names = list(set(player_stats["stats_name"].values.tolist()))

In [70]:
#matching_names = pd.DataFrame()
#matching_names["ID"] = transfers["Spieler"].apply(remove_accents).apply(remove_alphanumeric)
#matching_names['stats_name'] = matching_names["ID"].apply(lambda x: best_matching_name(x, stats_names))

In [71]:
#matching_names = matching_names.drop_duplicates()

In [72]:
#matching_names.to_csv("C:\\Users\\soere\\Bachelorarbeit\\datasets\\raw_data\\matching_names.csv",index=False)

In [73]:
#stats_clubs = list(set(player_stats["stats_club"].values.tolist()))

In [74]:
#matching_clubs = pd.DataFrame()
#matching_clubs["clubID"] = transfers["Abgebender Verein"].apply(remove_accents).apply(remove_alphanumeric)
#matching_clubs["stats_club"] = matching_clubs["clubID"].apply(lambda x: best_matching_name(x, stats_clubs))

In [75]:
#matching_clubs = matching_clubs.drop_duplicates()

In [76]:
#matching_clubs.to_csv("C:\\Users\\soere\\Bachelorarbeit\\datasets\\raw_data\\matching_clubs.csv",index=False)

Dasselbe für fifa datensatz

In [77]:
#fifa_names = list(set(fifa["fifa_name"].values.tolist()))

In [78]:
#matching_names_fifa = pd.DataFrame()
#matching_names_fifa["ID"] = transfers["Spieler"].apply(remove_accents).apply(remove_alphanumeric)
#matching_names_fifa['fifa_name'] = matching_names_fifa["ID"].apply(lambda x: best_matching_name(x, fifa_names))

In [79]:
#matching_names_fifa = matching_names_fifa.drop_duplicates()

In [80]:
#matching_names_fifa.to_csv("C:\\Users\\soere\\Bachelorarbeit\\datasets\\raw_data\\matching_names_fifa.csv",index=False)

In [81]:
#fifa_clubs = list(set(fifa["fifa_club"].values.tolist()))

In [82]:
#matching_clubs_fifa = pd.DataFrame()
#matching_clubs_fifa["clubID"] = transfers["Abgebender Verein"].apply(remove_accents).apply(remove_alphanumeric)
#matching_clubs_fifa["fifa_club"] = matching_clubs_fifa["clubID"].apply(lambda x: best_matching_name(x, fifa_clubs))

In [83]:
#matching_clubs_fifa = matching_clubs_fifa.drop_duplicates()

In [84]:
#matching_clubs_fifa.to_csv("C:\\Users\\soere\\Bachelorarbeit\\datasets\\raw_data\\matching_clubs_fifa.csv",index=False)

Merging

In [85]:
matching_names = pd.read_excel("C:\\Users\\soere\\Bachelorarbeit\\datasets\\raw_data\\matching_names.xlsx")

In [86]:
matching_clubs = pd.read_excel("C:\\Users\\soere\\Bachelorarbeit\\datasets\\raw_data\\matching_clubs.xlsx")

In [87]:
matching_names_fifa = pd.read_excel("C:\\Users\\soere\\Bachelorarbeit\\datasets\\raw_data\\matching_names_fifa.xlsx")

In [88]:
matching_clubs_fifa = pd.read_excel("C:\\Users\\soere\\Bachelorarbeit\\datasets\\raw_data\\matching_clubs_fifa.xlsx")

In [89]:
transfers_merged = pd.merge(transfers,matching_names,on=["ID"])

In [90]:
transfers_merged = pd.merge(transfers_merged,matching_clubs,on=["clubID"])

In [91]:
transfers_merged = pd.merge(transfers_merged,matching_names_fifa,on=["ID"])

In [92]:
transfers_merged = pd.merge(transfers_merged,matching_clubs_fifa,on=["clubID"])

In [93]:
transfers_merged = pd.merge(transfers_merged,player_stats,on=["stats_name","stats_club","Saison"])

In [94]:
transfers_merged = pd.merge(transfers_merged,fifa,on=["fifa_name","Saison"])

In [95]:
transfers_merged

,Saison,Spieler_x,Position_x,Alter_x,Marktwert,Nationalität_x,Abgebender Verein_x,Abgebende Liga,Aufnehmender Verein,Aufnehmende Liga,Ablöse,Transfer Typ,ID,clubID,stats_name,stats_club,fifa_name,fifa_club_x,Liga,Abgebender Verein_y,Spieler_y,Nationalität_y,Position_y,Alter_y,Geboren,born,MP,...,skill_ball_control,movement_acceleration,movement_sprint_speed,movement_agility,movement_reactions,movement_balance,power_shot_power,power_jumping,power_stamina,power_strength,power_long_shots,mentality_aggression,mentality_interceptions,mentality_positioning,mentality_vision,mentality_penalties,mentality_composure,defending_marking_awareness,defending_standing_tackle,defending_sliding_tackle,goalkeeping_diving,goalkeeping_handling,goalkeeping_kicking,goalkeeping_positioning,goalkeeping_reflexes,goalkeeping_speed,fifa_club_y
0,18,Kylian Mbappé,Centre-Forward,19,120.0,France,Monaco,Ligue 1,Paris SG,Ligue 1,180.0,Dauerhaft,kylianmbappe,monaco,kylianmbappe,monaco,kylianmbappelottin,monaco,FRA-Ligue 1,Monaco,Kylian Mbappé,FRA,"FW,MF",18.0,NaN,NaN,1,...,90,96,96,92,85,83,79,75,83,71,76,62,38,85,84,70,86.0,34,34,32,13,5,7,11,6,NaN,parissaintgermain
1,18,Riyad Mahrez,Right Winger,27,50.0,Algeria,Leicester,Premier League,Man City,Premier League,67.8,Dauerhaft,riyadmahrez,leicester,riyadmahrez,leicestercity,riyadmahrez,leicestercity,ENG-Premier League,Leicester City,Riyad Mahrez,ALG,"MF,FW",26.0,NaN,NaN,36,...,89,88,83,93,77,88,79,61,76,55,81,48,38,80,81,77,86.0,44,31,18,15,9,13,11,6,NaN,manchestercity
2,18,Thibaut Courtois,Goalkeeper,26,65.0,Belgium,Chelsea,Premier League,Real Madrid,LaLiga,35.0,Dauerhaft,thibautcourtois,chelsea,thibautcourtois,chelsea,thibautnicolasmarccourtois,chelsea,ENG-Premier League,Chelsea,Thibaut Courtois,BEL,GK,25.0,NaN,NaN,35,...,23,46,52,61,84,45,36,68,38,70,17,23,15,13,44,27,66.0,20,18,16,87,91,74,87,88,49.0,realmadrid
3,18,Alisson,Goalkeeper,25,60.0,Brazil,AS Roma,Serie A,Liverpool,Premier League,62.5,Dauerhaft,alisson,asroma,alisson,roma,alissonramsesbecker,roma,ITA-Serie A,Roma,Alisson,BRA,GK,24.0,NaN,NaN,37,...,30,54,45,40,83,37,21,52,32,78,14,27,11,13,66,23,65.0,15,19,16,82,81,85,83,88,49.0,liverpool
4,18,Gonzalo Higuaín,Centre-Forward,30,55.0,Argentina,Juventus,Serie A,AC Milan,Serie A,10.2,Dauerhaft,gonzalohiguain,juventus,gonzalohiguain,juventus,gonzalogerardohiguain,juventus,ITA-Serie A,Juventus,Gonzalo Higuaín,ARG,FW,29.0,NaN,NaN,35,...,85,73,73,75,86,69,86,79,70,85,80,50,20,92,74,70,86.0,35,22,18,5,12,7,5,10,NaN,milan
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3014,23,Tomas Suslov,Attacking Midfield,21,2.0,Slovakia,FC Groningen,Keuken Kampioen Divisie,Hellas Verona,Serie A,1.0,Dauerhaft,tomassuslov,fcgroningen,tomassuslov,groningen,tomassuslov,fcgroningen,Eredivisie,Groningen,Tomáš Suslov,SVK,"MF,FW",20.0,NaN,NaN,30,...,70,75,67,72,59,86,76,63,76,63,67,63,52,66,69,61,60.0,53,52,48,10,9,9,15,12,NaN,hellasverona
3015,23,Sergi Guardiola,Centre-Forward,32,2.0,Spain,Real Valladolid,2ª División,Cádiz CF,LaLiga,1.0,Dauerhaft,sergiguardiola,realvalladolid,sergiguardiola,valladolid,sergioguardiolanavarro,realvalladolid,ESP-La Liga,Valladolid,Sergi Guardiola,ESP,FW,31.0,NaN,NaN,14,...,74,68,72,73,61,66,73,81,70,78,70,70,28,74,65,66,65.0,32,19,18,11,14,11,11,11,NaN,cadiz
3016,23,Maximilian Mittelstädt,Left-Back,26,2.0,Germany,Hertha BSC,2. Bundesliga,VfB Stuttgart,Bundesliga,5.0,Dauerhaft,maximilianmittelstadt,herthabsc,maximilianmittelstadt,herthabsc,maximilianmittelstadt,herthabsc,GER-Bundesliga,Hertha BSC,Maximilian Mittelstädt,GER,"DF,MF",25.0,NaN,NaN,17,...,73,79,74,76,69,78,73,76,74,63,68,64,80,64,60,42,65.0,65,72,67,13,10,7,13,13,NaN,vfbstuttgart
3017,23,Maximilian Wittek,Left-Back,27,2.0,Germany,Vitesse,Eredivisie,VfL Bochum,Bundesliga,7.5,Dauerhaft,maximilianwittek,vitesse,maximilianwittek,vitesse,maximilianwittek,vitesse,Erediv

In [41]:
transfers_merged.loc[transfers_merged["Spieler_x"] == "Timo Werner"]

,Saison,Spieler_x,Position_x,Alter_x,Marktwert,Nationalität_x,Abgebender Verein_x,Abgebende Liga,Aufnehmender Verein,Aufnehmende Liga,Ablöse,Transfer Typ,ID,clubID,stats_name,stats_club,fifa_name,fifa_club_x,Liga,Abgebender Verein_y,Spieler_y,Nationalität_y,Position_y,Alter_y,Geboren,born,MP,...,skill_ball_control,movement_acceleration,movement_sprint_speed,movement_agility,movement_reactions,movement_balance,power_shot_power,power_jumping,power_stamina,power_strength,power_long_shots,mentality_aggression,mentality_interceptions,mentality_positioning,mentality_vision,mentality_penalties,mentality_composure,defending_marking_awareness,defending_standing_tackle,defending_sliding_tackle,goalkeeping_diving,goalkeeping_handling,goalkeeping_kicking,goalkeeping_positioning,goalkeeping_reflexes,goalkeeping_speed,fifa_club_y
1315,20,Timo Werner,Left Winger,24,64.0,Germany,RB Leipzig,Bundesliga,Chelsea,Premier League,53.0,Dauerhaft,timowerner,rbleipzig,timowerner,rbleipzig,timowerner,rbleipzig,GER-Bundesliga,RB Leipzig,Timo Werner,GER,FW,23.0,NaN,NaN,34,...,85,93,89,83,87,76,85,75,83,67,78,63,37,88,65,78,82.0,39,20,19,9,9,8,15,9,NaN,chelsea
2548,22,Timo Werner,Left Winger,26,35.0,Germany,Chelsea,Premier League,RB Leipzig,Bundesliga,20.0,Dauerhaft,timowerner,chelsea,timowerner,chelsea,timowerner,chelsea,ENG-Premier League,Chelsea,Timo Werner,GER,"FW,MF",25.0,NaN,NaN,21,...,83,93,88,79,85,76,85,75,81,64,77,63,37,87,68,78,79.0,39,20,19,9,9,8,15,9,NaN,rbleipzig
3411,23,Timo Werner,Left Winger,27,17.0,Germany,RB Leipzig,Bundesliga,Tottenham,Premier League,NaN,Leihe,timowerner,rbleipzig,timowerner,rbleipzig,timowerner,rbleipzig,GER-Bundesliga,RB Leipzig,Timo Werner,GER,"FW,MF",26.0,NaN,NaN,27,...,83,93,89,79,85,76,85,84,81,64,77,63,37,86,68,78,79.0,39,20,24,9,9,8,15,9,NaN,rbleipzig


In [96]:
transfers_merged["xGoalDiff"] = transfers_merged["Gls"] - transfers_merged["xG"]

In [97]:
transfers_merged.to_csv("C:\\Users\\soere\\Bachelorarbeit\\datasets\\raw_data\\test.csv",index=False)

In [98]:
transfers_merged.drop(columns=['ID', 'clubID',
       'stats_name', 'stats_club', 'Liga', 'Abgebender Verein_y', 'Spieler_y',
       'Nationalität_y', 'Position_y', 'Alter_y', 'Geboren', 'born','fifa_name', 'fifa_club_x','long_name',
       'club_name','fifa_club_y'],axis=1,inplace=True)

In [99]:
transfers_merged.rename(columns={"Spieler_x":"Spieler","Position_x":"Position","Alter_x":"Alter","Nationalität_x":"Nationalität","Abgebender Verein_x":"Abgebender Verein"},inplace=True)

In [100]:
print(transfers_merged.columns)

Index(['Saison', 'Spieler', 'Position', 'Alter', 'Marktwert', 'Nationalität',
       'Abgebender Verein', 'Abgebende Liga', 'Aufnehmender Verein',
       'Aufnehmende Liga', 'Ablöse', 'Transfer Typ', 'MP', 'Starts', 'Min',
       '90s', 'Gls', 'Ast', 'G+A', 'G-PK', 'PK', 'PKatt', 'CrdY', 'CrdR', 'xG',
       'npxG', 'xAG', 'npxG+xAG', 'PrgC', 'PrgP', 'Gls p90', 'Ast p90',
       'G+A p90', 'G-PK p90', 'G+A-PK p90', 'xG p90', 'xAG p90', 'xG+xAG p90',
       'npxG p90', 'npxG+xAG p90', 'npxG+xAG.1', 'overall', 'potential',
       'preferred_foot', 'weak_foot', 'skill_moves',
       'international_reputation', 'work_rate', 'body_type', 'pace',
       'shooting', 'passing', 'dribbling', 'defending', 'physic',
       'attacking_crossing', 'attacking_finishing',
       'attacking_heading_accuracy', 'attacking_short_passing',
       'attacking_volleys', 'skill_dribbling', 'skill_curve',
       'skill_fk_accuracy', 'skill_long_passing', 'skill_ball_control',
       'movement_acceleration', 'mo

In [101]:
transfers_merged.to_csv("C:\\Users\\soere\\Bachelorarbeit\\datasets\\transfers_dataset_merged.csv",index=False)

In [102]:
transfers_merged

,Saison,Spieler,Position,Alter,Marktwert,Nationalität,Abgebender Verein,Abgebende Liga,Aufnehmender Verein,Aufnehmende Liga,Ablöse,Transfer Typ,MP,Starts,Min,90s,Gls,Ast,G+A,G-PK,PK,PKatt,CrdY,CrdR,xG,npxG,xAG,...,skill_ball_control,movement_acceleration,movement_sprint_speed,movement_agility,movement_reactions,movement_balance,power_shot_power,power_jumping,power_stamina,power_strength,power_long_shots,mentality_aggression,mentality_interceptions,mentality_positioning,mentality_vision,mentality_penalties,mentality_composure,defending_marking_awareness,defending_standing_tackle,defending_sliding_tackle,goalkeeping_diving,goalkeeping_handling,goalkeeping_kicking,goalkeeping_positioning,goalkeeping_reflexes,goalkeeping_speed,xGoalDiff
0,18,Kylian Mbappé,Centre-Forward,19,120.0,France,Monaco,Ligue 1,Paris SG,Ligue 1,180.0,Dauerhaft,1,1.0,74.0,0.8,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.2,0.2,0.0,...,90,96,96,92,85,83,79,75,83,71,76,62,38,85,84,70,86.0,34,34,32,13,5,7,11,6,NaN,-0.2
1,18,Riyad Mahrez,Right Winger,27,50.0,Algeria,Leicester,Premier League,Man City,Premier League,67.8,Dauerhaft,36,34.0,2950.0,32.8,12.0,10.0,22.0,12.0,0.0,0.0,2.0,0.0,6.5,6.5,7.2,...,89,88,83,93,77,88,79,61,76,55,81,48,38,80,81,77,86.0,44,31,18,15,9,13,11,6,NaN,5.5
2,18,Thibaut Courtois,Goalkeeper,26,65.0,Belgium,Chelsea,Premier League,Real Madrid,LaLiga,35.0,Dauerhaft,35,35.0,3150.0,35.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,...,23,46,52,61,84,45,36,68,38,70,17,23,15,13,44,27,66.0,20,18,16,87,91,74,87,88,49.0,0.0
3,18,Alisson,Goalkeeper,25,60.0,Brazil,AS Roma,Serie A,Liverpool,Premier League,62.5,Dauerhaft,37,37.0,3330.0,37.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,30,54,45,40,83,37,21,52,32,78,14,27,11,13,66,23,65.0,15,19,16,82,81,85,83,88,49.0,0.0
4,18,Gonzalo Higuaín,Centre-Forward,30,55.0,Argentina,Juventus,Serie A,AC Milan,Serie A,10.2,Dauerhaft,35,32.0,2815.0,31.3,16.0,5.0,21.0,15.0,1.0,2.0,4.0,0.0,11.8,10.2,3.4,...,85,73,73,75,86,69,86,79,70,85,80,50,20,92,74,70,86.0,35,22,18,5,12,7,5,10,NaN,4.2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3014,23,Tomas Suslov,Attacking Midfield,21,2.0,Slovakia,FC Groningen,Keuken Kampioen Divisie,Hellas Verona,Serie A,1.0,Dauerhaft,30,26.0,2123.0,23.6,1.0,1.0,2.0,1.0,0.0,0.0,6.0,0.0,2.3,2.3,2.7,...,70,75,67,72,59,86,76,63,76,63,67,63,52,66,69,61,60.0,53,52,48,10,9,9,15,12,NaN,-1.3
3015,23,Sergi Guardiola,Centre-Forward,32,2.0,Spain,Real Valladolid,2ª División,Cádiz CF,LaLiga,1.0,Dauerhaft,14,8.0,502.0,5.6,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0,0.1,...,74,68,72,73,61,66,73,81,70,78,70,70,28,74,65,66,65.0,32,19,18,11,14,11,11,11,NaN,-1.0
3016,23,Maximilian Mittelstädt,Left-Back,26,2.0,Germany,Hertha BSC,2. Bundesliga,VfB Stuttgart,Bundesliga,5.0,Dauerhaft,17,7.0,849.0,9.4,0.0,0.0,0.0,0.0,0.0,0.0,5.0,0.0,0.3,0.3,0.8,...,73,79,74,76,69,78,73,76,74,63,68,64,80,64,60,42,65.0,65,72,67,13,10,7,13,13,NaN,-0.3
3017,23,Maximilian Wittek,Left-Back,27,2.0,Germany,Vitesse,Eredivisie,VfL Bochum,Bundesliga,7.5,Dauerhaft,31,31.0,2715.0,30.2,3.0,7.0,10.0,3.0,0.0,0.0,9.0,1.0,3.0,3.0,6.8,...,70,77,78,79,71,85,80,73,79,68,71,77,66,71,58,52,72.0,65,69,71,15,12,12,13,7,NaN,0.0


In [103]:
transfers

,Saison,Spieler,Position,Alter,Marktwert,Nationalität,Abgebender Verein,Abgebende Liga,Aufnehmender Verein,Aufnehmende Liga,Ablöse,Transfer Typ,ID,clubID
0,18,Kylian Mbappé,Centre-Forward,19,120.0,France,Monaco,Ligue 1,Paris SG,Ligue 1,180.0,Dauerhaft,kylianmbappe,monaco
1,18,Kepa Arrizabalaga,Goalkeeper,23,20.0,Spain,Athletic,LaLiga,Chelsea,Premier League,80.0,Dauerhaft,kepaarrizabalaga,athletic
2,18,Riyad Mahrez,Right Winger,27,50.0,Algeria,Leicester,Premier League,Man City,Premier League,67.8,Dauerhaft,riyadmahrez,leicester
3,18,Thibaut Courtois,Goalkeeper,26,65.0,Belgium,Chelsea,Premier League,Real Madrid,LaLiga,35.0,Dauerhaft,thibautcourtois,chelsea
4,18,Alisson,Goalkeeper,25,60.0,Brazil,AS Roma,Serie A,Liverpool,Premier League,62.5,Dauerhaft,alisson,asroma
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11970,23,Tomas Suslov,Attacking Midfield,21,2.0,Slovakia,FC Groningen,Keuken Kampioen Divisie,Hellas Verona,Serie A,1.0,Dauerhaft,tomassuslov,fcgroningen
11971,23,Sergi Guardiola,Centre-Forward,32,2.0,Spain,Real Valladolid,2ª División,Cádiz CF,LaLiga,1.0,Dauerhaft,sergiguardiola,realvalladolid
11979,23,Maximilian Mittelstädt,Left-Back,26,2.0,Germany,Hertha BSC,2. Bundesliga,VfB Stuttgart,Bundesliga,5.0,Dauerhaft,maximilianmittelstadt,herthabsc
11988,23,Maximilian Wittek,Left-Back,27,2.0,Germany,Vitesse,Eredivisie,VfL Bochum,Bundesliga,7.5,Dauerhaft,maximilianwittek,vitesse
